In [1]:
!pip install lxml
!pip install html5lib
!pip install beautifulsoup4
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
import requests 
from pandas.io.json import json_normalize
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
!wget -q -O 'Canada_Data.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


In [3]:
from bs4 import BeautifulSoup
import requests
with open('Canada_Data.html') as html_file:
    soup = BeautifulSoup(html_file,'lxml')
    #print(soup.prettify())

In [4]:
table = soup.find('table',class_='wikitable sortable')
#print(table.prettify())
data = []
for th in table.tbody.tr.find_all('th'):
        column_name = th.text
        data.append(column_name)
        print(column_name)
        print(data)

Postcode
['Postcode']
Borough
['Postcode', 'Borough']
Neighbourhood

['Postcode', 'Borough', 'Neighbourhood\n']


In [5]:
import pandas as pd
Postal_Codes = pd.DataFrame(columns=data)
Postal_Codes


,Postcode,Borough,Neighbourhood


In [6]:
data1 = []
for td in table.find_all('td'):
         column_values = td.text
         data1.append(column_values)#print(column_values)         

In [7]:
length = len(data1) 
print(length)
Postcode = []
Borough = []
Neighbourhood = []

for i in range(0,length,3): 
    #print(i,data1[i])
    Postcode.append(data1[i])

864


In [8]:
for i in range(1,length,3): 
    #print(i,data1[i])
    Borough.append(data1[i])

In [9]:
for i in range(2,length,3): 
    #print(i,data1[i])
    Neighbourhood.append(data1[i])
    

In [10]:
Neighbourhood1 = Neighbourhood
Neighbourhood[:] = [s.replace('\n', '') for s in Neighbourhood]
#Neighbourhood
df = pd.DataFrame(list(zip(Postcode, Borough, Neighbourhood)), 
               columns =['Postcode', 'Borough','Neighbourhood']) 
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [11]:
df1 = df.loc[df['Borough'] != 'Not assigned']
df1['Neighbourhood'] = df1['Borough'].where(df1['Neighbourhood'] == 'Not assigned',df1['Neighbourhood'])
df2 = df1.groupby(['Postcode', 'Borough'],as_index=False)['Neighbourhood'].apply(','.join).to_frame('Neighbourhood').reset_index()
df2

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [12]:
Neighbourhood_NEW = []
Neighbourhood_NEW = df2['Neighbourhood']
Neighbourhood_NEW[:] = [s.replace('Canada Post Gateway Processing Centre', 'Mississauga') for s in Neighbourhood_NEW]
Neighbourhood_NEW[:] = [s.replace('Stn A PO Boxes 25 The Esplanade', 'Downtown Toronto') for s in Neighbourhood_NEW]
Neighbourhood_NEW[:] = [s.replace('CFB Toronto', 'Canadian Forces base') for s in Neighbourhood_NEW]
Neighbourhood_NEW[:] = [s.replace('Business Reply Mail Processing Centre 969 Eastern', 'East Toronto') for s in Neighbourhood_NEW]

Neighbourhood_NEW

0                                          Rouge,Malvern
1                   Highland Creek,Rouge Hill,Port Union
2                        Guildwood,Morningside,West Hill
3                                                 Woburn
4                                              Cedarbrae
5                                    Scarborough Village
6              East Birchmount Park,Ionview,Kennedy Park
7                          Clairlea,Golden Mile,Oakridge
8          Cliffcrest,Cliffside,Scarborough Village West
9                             Birch Cliff,Cliffside West
10     Dorset Park,Scarborough Town Centre,Wexford He...
11                                      Maryvale,Wexford
12                                             Agincourt
13                 Clarks Corners,Sullivan,Tam O'Shanter
14     Agincourt North,L'Amoreaux East,Milliken,Steel...
15                                       L'Amoreaux West
16                                           Upper Rouge
17                             

In [13]:
length1 = len(Neighbourhood_NEW)
length1
Latitude = []
Longitude = []
for i in range(0,length1,1): 
    address = Neighbourhood_NEW[i].split(',')[:1]
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(Neighbourhood_NEW[i].split(',')[:1])
    latitude = location.latitude
    longitude = location.longitude
    Latitude.append(latitude)
    Longitude.append(longitude)

In [14]:
df2['Latitude'] = Latitude
df2['Longitude'] = Longitude
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",48.283710,0.705539
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.790117,-79.173334
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.754899,-79.197776
3,M1G,Scarborough,Woburn,42.479262,-71.152277
4,M1H,Scarborough,Cedarbrae,50.956318,-114.129323


#### Exploring Toronto with neighborhoods 

In [15]:
Toronto_data = df2[df2['Borough'].str.contains("Toronto")==True].reset_index(drop=True)
Toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.671024,-79.296712
1,M4K,East Toronto,"The Danforth West,Riverdale",43.686360,-79.300316
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.671024,-79.296712
3,M4M,East Toronto,Studio District,32.331123,34.851543
4,M4N,Central Toronto,Lawrence Park,43.729199,-79.403252


In [16]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude1 = location.latitude
longitude1 = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude1, longitude1))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [17]:
map_toronto = folium.Map(location=[latitude1, longitude1], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [18]:
CLIENT_ID = 'MTHT3ZUA4FMUAB4KN5KSCQEJEFENE5UGOBVFZ52FXWHHUS1C' # your Foursquare ID
CLIENT_SECRET = 'GRKZYWARSKW05FUDQFCYIJSAZTTDQNGAXTN1IV1KOSEPAQPE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MTHT3ZUA4FMUAB4KN5KSCQEJEFENE5UGOBVFZ52FXWHHUS1C
CLIENT_SECRET:GRKZYWARSKW05FUDQFCYIJSAZTTDQNGAXTN1IV1KOSEPAQPE


#### Function to explore all the neighborhoods in Toronto

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        LIMIT = 100 # limit of number of venues returned by Foursquare API
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

In [ ]:
Toronto_venues.head()

In [ ]:
Toronto_venues.groupby('Neighbourhood').count()

####  Analyzing Each Neighborhood

In [ ]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

In [ ]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

##### Toronto neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

####  Cluster Neighborhoods

In [ ]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighbourhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters